In [29]:
import sys
sys.path.append('/home/meiri.yoav/biomed_proj/pyPPG')

from Prefiltering import*
from FiducialPoints import*
from Biomarkers2 import*
from Summary import*
from Statistics import*
import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
# set seed 42
np.random.seed(42)

Setup:

Input: 10 ppg cycles (each one represented as 1x30 feature vector, total 10x30 matrix)

Output: 0/1 (0: non_af, 1: af)



In [30]:
def sample_series(X, B, series_size):
    n = X.shape[0]
    idx = np.random.randint(0, n-series_size, B)
    onsets = np.zeros(B)
    X_sampled = np.zeros((B, series_size, X.shape[1]-3)) # substracting 3 because we ommit Tpi, sample_idx and os
    for i in range(B):
        X_sampled[i,:,:] = X.iloc[idx[i]:idx[i]+series_size,:].drop(columns=['Tpi', 'sample_idx', 'os'])
        # put in onsets[i] the onset value of the record in idx[i]
        onsets[i] = X.iloc[idx[i],:].os
        
    return X_sampled, onsets, idx


In this setup we train on sampled 30 cycle long series and test & validate on non-sampled (sliced) 30 cycle long series.

In [31]:
def excract_hrv_features(cycle_signal_30, fs):
    from hrvanalysis.extract_features import get_time_domain_features
    from hrvanalysis.extract_features import get_frequency_domain_features
    from hrvanalysis.extract_features import get_geometrical_features
    from hrvanalysis.extract_features import get_poincare_plot_features
    from hrvanalysis.extract_features import get_csi_cvi_features
    import pandas as pd

    ''' reference: https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html'''

    time_domain_features = get_time_domain_features(cycle_signal_30)
    frequency_domain_features = get_frequency_domain_features(cycle_signal_30, sampling_frequency=fs)
    # geometrical_features = get_geometrical_features(cycle_signal_30)
    poincare_plot_features = get_poincare_plot_features(cycle_signal_30)
    csi_cvi_features = get_csi_cvi_features(cycle_signal_30)

    # aggregate all features into one dict and turn it into a dataframe
    features = {
                **time_domain_features,
                **frequency_domain_features,
                # **geometrical_features,
                **poincare_plot_features,
                **csi_cvi_features
                }
    features_arr = pd.DataFrame(features, index=[0]).to_numpy()
    return features_arr.squeeze()

def get_outlier_bounds(arr):
    q1 = np.quantile(arr, 0.25)
    q3 = np.quantile(arr, 0.75)
    iqr = q3-q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    return lower_bound, upper_bound


In [32]:
import random
from sklearn.impute import KNNImputer
from hrvanalysis.preprocessing import get_nn_intervals

B = 100# 1000 10 cycle long samples for each subject
series_size = 30
fs = 125
add_hrv = False
add_morph = True

annot_files = sorted(glob.glob("/home/meiri.yoav/biomed_proj/data/annotated/*"))
af_files = [f for f in annot_files if 'non' not in f]
non_af_files = [f for f in annot_files if 'non' in f]

names = [tuple(f.split('/')[-1].split('.')[0].split('_')[1:]) for f in annot_files]
non_af_subs = [t  for t in list(names) if 'non' in t]
af_subs = [t  for t in list(names) if 'non' not in t]

train_subs = non_af_subs[:int(len(non_af_subs)*0.8)] + af_subs[:int(len(af_subs)*0.8)]
test_subs = non_af_subs[int(len(non_af_subs)*0.8):] + af_subs[int(len(af_subs)*0.8):]

# print how much af and non af subjects we have in train and test sets
print(f'af subjects in train set: {len([s for s in train_subs if "non" not in s])}')
print(f'non af subjects in train set: {len([s for s in train_subs if "non" in s])}')
print(f'af subjects in test set: {len([s for s in test_subs if "non" not in s])}')
print(f'non af subjects in test set: {len([s for s in test_subs if "non" in s])}')

subs_features = {}
for f in annot_files:
    sub_name = tuple(f.split('/')[-1].split('.')[0].split('_')[1:])
    # print(f'processing {sub_name}')
    subs_features[sub_name] = {}
    with open(f, 'rb') as f:
        annot = pkl.load(f)

    
    X = annot['osignal_data']


    if sub_name in train_subs:
        # sample a random number between 0 and 
        # upper_bound = random.randint(0, int(len(X)/5))
        test_subset = X.iloc[0:int(len(X)/5),:] # take the first 20% of the data as test subset
        
        first_index_of_X = int(len(X)/5)
        # remove test subset from X
        X = X.drop(test_subset.index)
        
        
        signal = annot['hr']
        peaks = annot['fiducials']['pk']
        # # compute lower and upper bound that can be used for outlier removal
        # lower_bound, upper_bound = get_outlier_bounds(raw_signal)
        # signal = get_nn_intervals(annot['hr'], low_rri=lower_bound, high_rri=upper_bound)
        
        #? Setup test and validation sets
        # devide test subset to 30 cycle long series and pad with zeros if needed
        test_series = np.zeros((int(len(test_subset)/series_size), series_size, test_subset.shape[1]-3)) # substracting 3 because we ommit Tpi, sample_idx and os
        # test_onsets = []
        hrv_features = []
        for i in range(int(len(test_subset)/series_size)):
            # # put in test_onsets[i] the onset value of the record in idx[i]
            # if i == int(len(test_subset)/series_size)-1:
            #     test_onsets.append((test_subset.iloc[i*series_size,:].os, X.iloc[0,:].os)) # the ending of the last series is the beginning of the first series in X
            # else:
            #     test_onsets.append((test_subset.iloc[i*series_size,:].os, test_subset.iloc[(i+1)*series_size-1,:].os))
            
            # take peaks[i*series_size:(i+1)*series_size] and substract from each element it's previous element 
            if add_hrv:
                right_bound = min((i+1)*series_size+1, len(peaks))
                nn_intervals = np.diff(peaks[i*series_size : right_bound])
                # print(i, ': ', nn_intervals.shape, excract_hrv_features(nn_intervals, fs).shape)
                hrv_features.append(excract_hrv_features(nn_intervals, fs))
            
            if add_morph:
                test_series[i,:,:] = test_subset.iloc[i*series_size:(i+1)*series_size,:].drop(columns=['Tpi', 'sample_idx', 'os'])
        
        # print(np.array(hrv_features).shape)
        if add_hrv:
            # add hrv features to test_series
            hrv_features = np.array([hrv_features for i in range(series_size)]).reshape((len(hrv_features), series_size, hrv_features[0].shape[0]))
            test_series = np.concatenate((test_series, hrv_features), axis=2)
        
        # if add_hrv:
        #     # extract hrv features from test_series
        #     test_hrv_features = np.nan_to_num(np.vstack(list(map(lambda x: excract_hrv_features(signal[int(x[0]):int(x[1])], fs), test_onsets))))
        #     # add hrv features to test_series
        #     test_hrv_features = np.array([test_hrv_features for i in range(series_size)]).reshape((test_hrv_features.shape[0], series_size, test_hrv_features.shape[1]))
        #     test_series = np.concatenate((test_series, test_hrv_features), axis=2)

        # split test_series to val split and test split and add them both to subs_features
        subs_features[sub_name]['val_split'] = test_series[:int(len(test_series)/2)]
        subs_features[sub_name]['test_split'] = test_series[int(len(test_series)/2):]
        
        #? Setup train set
        
        data, onsets, idx = sample_series(X, B, series_size)

        if not add_morph:
            # make data an array of zeros of the same shape as data
            data = np.zeros(data.shape)
        # print(data.shape)
        if add_hrv:
            hrv_features = []
            for i in range(B):
                right_bound = min(first_index_of_X+idx[i]+series_size+1, len(peaks))
                nn_intervals = np.diff(peaks[first_index_of_X+idx[i] : right_bound])
                hrv_features.append(excract_hrv_features(nn_intervals, fs))
                
            hrv_features = np.array([hrv_features for i in range(series_size)]).reshape((len(hrv_features), series_size, hrv_features[0].shape[0]))
            data = np.concatenate((data, hrv_features), axis=2)
            
            # train_onsets = [(onsets[i], onsets[i+1]) if i < len(onsets)-1 else (onsets[i], len(signal)) for i in range(len(onsets)-1)]
            # # extract hrv features from train_series
            # train_hrv_features= np.nan_to_num(np.vstack(list(map(lambda x: excract_hrv_features(signal[int(x[0]):int(x[1])], fs), train_onsets))))
            # # add hrv features to train_series
            # train_hrv_features = np.array([train_hrv_features for i in range(series_size)]).reshape((train_hrv_features.shape[0], series_size, train_hrv_features.shape[1])) # to all series vectors add the same hrv features (because the characterize the whole record and not every beat)
            # data = np.concatenate((data, train_hrv_features), axis=2)
        
        
        subs_features[sub_name]['train_split'] = data
    else:
        # print('test_sub')
        # In this case just split to 30 cycle long series and pad with zeros if needed
        data = np.zeros((int(len(X)/series_size), series_size, X.shape[1]-3))
        # test_onsets = []
        hrv_features = []
        for i in range(int(len(X)/series_size)):
            # if i == int(len(test_subset)/series_size)-1:
            #     test_onsets.append((X.iloc[i*series_size,:].os, len(signal)))
            # else:
            #     test_onsets.append((X.iloc[i*series_size,:].os, X.iloc[(i+1)*series_size-1,:].os))
            
            if add_hrv:
                right_bound = min((i+1)*series_size+1, len(peaks))
                if right_bound <= i*series_size:
                    continue
                # print((i*series_size, right_bound))
                nn_intervals = np.diff(peaks[i*series_size : right_bound])
                if len(nn_intervals) == 0:
                    continue
                hrv_features.append(excract_hrv_features(nn_intervals, fs))
            if add_morph:
                data[i,:,:] = X.iloc[i*series_size:(i+1)*series_size,:].drop(columns=['Tpi', 'sample_idx', 'os'])
        
        if add_hrv:
            # add hrv features to test_series
            hrv_features = np.array([hrv_features for i in range(series_size)]).reshape((len(hrv_features), series_size, hrv_features[0].shape[0]))
            # print(hrv_features.shape)
            data = np.concatenate((data[:len(hrv_features),:,:], hrv_features), axis=2)
        
        # if add_hrv:
        #     # extract hrv features from test_series
        #     test_hrv_features = np.nan_to_num(np.vstack(list(map(lambda x: excract_hrv_features(signal[int(x[0]):int(x[1])], fs), test_onsets))))
        #     # add hrv features to test_series
        #     test_hrv_features = np.array([test_hrv_features for i in range(series_size)]).reshape((test_hrv_features.shape[0],series_size, test_hrv_features.shape[1]))
        #     data = np.concatenate((data, test_hrv_features), axis=2)
        # split data to val split and test split and add them both to subs_features
        subs_features[sub_name]['val_split'] = data[:int(len(data)/2)]
        subs_features[sub_name]['test_split'] = data[int(len(data)/2):]
        

# print the name and the shapes of all subs in subs_features, and consider if it's a train sub or a test sub
for sub in subs_features.keys():
    if sub in train_subs:
        print(sub, subs_features[sub]['train_split'].shape, subs_features[sub]['val_split'].shape, subs_features[sub]['test_split'].shape)
    else:
        print(sub, subs_features[sub]['val_split'].shape, subs_features[sub]['test_split'].shape)


af subjects in train set: 14
non af subjects in train set: 12
af subjects in test set: 4
non af subjects in test set: 3
('af', '001') (100, 30, 30) (5, 30, 30) (6, 30, 30)
('af', '002') (100, 30, 30) (4, 30, 30) (5, 30, 30)
('af', '003') (100, 30, 30) (4, 30, 30) (5, 30, 30)
('af', '005') (100, 30, 30) (4, 30, 30) (4, 30, 30)
('af', '006') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '007') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '008') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '009') (100, 30, 30) (5, 30, 30) (6, 30, 30)
('af', '010') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '011') (100, 30, 30) (5, 30, 30) (6, 30, 30)
('af', '012') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '013') (100, 30, 30) (5, 30, 30) (5, 30, 30)
('af', '014') (100, 30, 30) (5, 30, 30) (6, 30, 30)
('af', '015') (100, 30, 30) (4, 30, 30) (5, 30, 30)
('af', '016') (29, 30, 30) (29, 30, 30)
('af', '017') (24, 30, 30) (24, 30, 30)
('af', '018') (29, 30, 30) (30, 30, 30)
('af', '019') (27, 30, 30) (28, 

In [33]:
print(train_subs)

[('non', 'af', '001'), ('non', 'af', '002'), ('non', 'af', '003'), ('non', 'af', '005'), ('non', 'af', '006'), ('non', 'af', '007'), ('non', 'af', '008'), ('non', 'af', '009'), ('non', 'af', '010'), ('non', 'af', '011'), ('non', 'af', '012'), ('non', 'af', '013'), ('af', '001'), ('af', '002'), ('af', '003'), ('af', '005'), ('af', '006'), ('af', '007'), ('af', '008'), ('af', '009'), ('af', '010'), ('af', '011'), ('af', '012'), ('af', '013'), ('af', '014'), ('af', '015')]


In [34]:
def create_dataset(sub_features):
    # devide to train and test subjects where both train and test groups contain all the classes
    
    
    X_train = np.concatenate([sub_features[t]['train_split'] for t in train_subs], axis=0) # amoung the train subjects, take 80% of each subject to train set
    y_train = np.concatenate([np.ones(int(sub_features[t]['train_split'].shape[0]))*int('non' not in t) for t in train_subs], axis=0)
    
    X_val_from_train_subs = np.concatenate([sub_features[t]['val_split'] for t in train_subs], axis=0)
    y_val_from_train_subs = np.concatenate([np.ones(int(sub_features[t]['val_split'].shape[0]))*int('non' not in t) for t in train_subs], axis=0)
    # shuffle X_val_from_train_subs and y_val_from_train_subs togather
    idx = np.random.permutation(X_val_from_train_subs.shape[0])
    X_val_from_train_subs = X_val_from_train_subs[idx]
    y_val_from_train_subs = y_val_from_train_subs[idx]
    
    X_test_from_train_subs = np.concatenate([sub_features[t]['test_split'] for t in train_subs], axis=0)
    y_test_from_train_subs = np.concatenate([np.ones(int(sub_features[t]['test_split'].shape[0]))*int('non' not in t) for t in train_subs], axis=0)
    # print(y_test_from_train_subs.shape, X_test_from_train_subs.shape)
    # for i in range(len(y_test_from_train_subs)):
    #     print(y_test_from_train_subs[i], X_test_from_train_subs[i].shape)
    X_val_from_test_subs = np.concatenate([sub_features[t]['val_split'] for t in test_subs], axis=0)
    y_val_from_test_subs = np.concatenate([np.ones(int(sub_features[t]['val_split'].shape[0]))*int('non' not in t) for t in test_subs], axis=0)
    
    X_test_from_test_subs = np.concatenate([sub_features[t]['test_split'] for t in test_subs], axis=0)
    y_test_from_test_subs = np.concatenate([np.ones(int(sub_features[t]['test_split'].shape[0]))*int('non' not in t) for t in test_subs], axis=0)
    
    
    return X_train, y_train, X_val_from_train_subs, y_val_from_train_subs, X_test_from_train_subs, y_test_from_train_subs, X_val_from_test_subs, y_val_from_test_subs, X_test_from_test_subs, y_test_from_test_subs, train_subs, test_subs
    

    
X_train, y_train, X_val_from_train_subs, y_val_from_train_subs, X_test_from_train_subs, y_test_from_train_subs, X_val_from_test_subs, y_val_from_test_subs, X_test_from_test_subs, y_test_from_test_subs, train_subs, test_subs = create_dataset(subs_features)
# print the shapes of the data
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_val_from_train_subs shape: ', X_val_from_train_subs.shape)
print('y_val_from_train_subs shape: ', y_val_from_train_subs.shape)
print('X_test_from_train_subs shape: ', X_test_from_train_subs.shape)
print('y_test_from_train_subs shape: ', y_test_from_train_subs.shape)
print('X_val_from_test_subs shape: ', X_val_from_test_subs.shape)
print('y_val_from_test_subs shape: ', y_val_from_test_subs.shape)
print('X_test_from_test_subs shape: ', X_test_from_test_subs.shape)
print('y_test_from_test_subs shape: ', y_test_from_test_subs.shape)

print('train_subs: ', train_subs)
print('test_subs: ', test_subs)

X_train shape:  (2600, 30, 30)
y_train shape:  (2600,)
X_val_from_train_subs shape:  (122, 30, 30)
y_val_from_train_subs shape:  (122,)
X_test_from_train_subs shape:  (135, 30, 30)
y_test_from_train_subs shape:  (135,)
X_val_from_test_subs shape:  (184, 30, 30)
y_val_from_test_subs shape:  (184,)
X_test_from_test_subs shape:  (187, 30, 30)
y_test_from_test_subs shape:  (187,)
train_subs:  [('non', 'af', '001'), ('non', 'af', '002'), ('non', 'af', '003'), ('non', 'af', '005'), ('non', 'af', '006'), ('non', 'af', '007'), ('non', 'af', '008'), ('non', 'af', '009'), ('non', 'af', '010'), ('non', 'af', '011'), ('non', 'af', '012'), ('non', 'af', '013'), ('af', '001'), ('af', '002'), ('af', '003'), ('af', '005'), ('af', '006'), ('af', '007'), ('af', '008'), ('af', '009'), ('af', '010'), ('af', '011'), ('af', '012'), ('af', '013'), ('af', '014'), ('af', '015')]
test_subs:  [('non', 'af', '014'), ('non', 'af', '015'), ('non', 'af', '016'), ('af', '016'), ('af', '017'), ('af', '018'), ('af', '0

In [35]:
# check if there is nan values in the data
print('X_train nan: ', np.isnan(X_train).any())
print('y_train nan: ', np.isnan(y_train).any())
print('X_val_from_train_subs nan: ', np.isnan(X_val_from_train_subs).any())
print('y_val_from_train_subs nan: ', np.isnan(y_val_from_train_subs).any())
print('X_test_from_train_subs nan: ', np.isnan(X_test_from_train_subs).any())
print('y_test_from_train_subs nan: ', np.isnan(y_test_from_train_subs).any())
print('X_val_from_test_subs nan: ', np.isnan(X_val_from_test_subs).any())
print('y_val_from_test_subs nan: ', np.isnan(y_val_from_test_subs).any())
print('X_test_from_test_subs nan: ', np.isnan(X_test_from_test_subs).any())
print('y_test_from_test_subs nan: ', np.isnan(y_test_from_test_subs).any())


X_train nan:  False
y_train nan:  False
X_val_from_train_subs nan:  False
y_val_from_train_subs nan:  False
X_test_from_train_subs nan:  False
y_test_from_train_subs nan:  False
X_val_from_test_subs nan:  False
y_val_from_test_subs nan:  False
X_test_from_test_subs nan:  False
y_test_from_test_subs nan:  False


In [36]:
import wandb
wandb.login()
user = "YoavMeiri"
project = "bioML - AF detection from PPG - lstm baseline"
name = "only Morph, B=100"
run = wandb.init(entity=user, project=project, name=name, config={
    "architecture": 'biderctional lstm',
    })


In [37]:
# create a dataset and model using pytorch lightning module and biderectional lstm for this task
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn.functional as F
from sklearn.metrics import classification_report
from pytorch_lightning.loggers import WandbLogger
import torchmetrics

wandb_logger = WandbLogger()


class afPPGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y
        self.len = len(y)
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len

class afPPGModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, learning_rate):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        # Add 2 layer fully-connected with ReLu in the middle
        self.fc = nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
            nn.Softmax())
        
        self.learning_rate = learning_rate
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1 = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)
        self.auc = torchmetrics.AUROC(task="multiclass", num_classes=num_classes)
        self.save_hyperparameters()
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    
    def training_step(self, batch, batch_idx):
        loss = self._log_stats(batch,  name_template='train')

        return loss
    
    def validation_step(self, batch, batch_idx, dataloader_idx):
        if dataloader_idx == 0:
            loss = self._log_stats(batch,  name_template='val_train_subs')

        elif dataloader_idx == 1:
            loss = self._log_stats(batch,  name_template='val_test_subs')
        return loss
    
    def test_step(self, batch, batch_idx, dataloader_idx):
        if dataloader_idx == 0:
            loss = self._log_stats(batch,  name_template='test_train_subs')
        else:
            loss = self._log_stats(batch,  name_template='test_test_subs')
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def train_dataloader(self):
        train_dataset = afPPGDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        return train_loader
    
    def val_dataloader(self):
        val_from_train_subs_dataset = afPPGDataset(X_val_from_train_subs, y_val_from_train_subs)
        val_from_train_subs_loader = DataLoader(val_from_train_subs_dataset, batch_size=32, shuffle=False)
        
        val_from_test_subs_dataset = afPPGDataset(X_val_from_test_subs, y_val_from_test_subs)
        val_from_test_subs_loader = DataLoader(val_from_test_subs_dataset, batch_size=32, shuffle=False)
        return [val_from_train_subs_loader, val_from_test_subs_loader]
    
    def test_dataloader(self):
        test_from_train_subs_dataset = afPPGDataset(X_test_from_train_subs, y_test_from_train_subs)
        test_from_train_subs_loader = DataLoader(test_from_train_subs_dataset, batch_size=32, shuffle=False)
        
        test_from_test_subs_dataset = afPPGDataset(X_test_from_test_subs, y_test_from_test_subs)
        test_from_test_subs_loader = DataLoader(test_from_test_subs_dataset, batch_size=32, shuffle=False)
        return [test_from_train_subs_loader, test_from_test_subs_loader]
    
    def _log_stats(self, batch, name_template):
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = F.cross_entropy(logits, y.long())
        f1 = self.f1(preds, y)
        acc = self.accuracy(preds, y)
        
        self.log(name_template + '_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log(name_template + '_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log(name_template + '_f1', f1, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def predict(self, x):
        y_hat = self(x)
        return y_hat
    
    def predict_proba(self, x):
        y_hat = self(x)
        return F.softmax(y_hat, dim=1)

# create a model
input_dim = 59 if add_hrv else 30
model = afPPGModel(input_size=input_dim, hidden_size=64, num_layers=2, num_classes=2, learning_rate=0.0001)

# create a trainer
# trainer = pl.Trainer(max_epochs=15)
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, num_sanity_val_steps=2, accelerator='cpu')

trainer.validate(model)
# train the model
trainer.fit(model)

# test the model
trainer.test(model)

wandb.finish()



/home/meiri.yoav/mambaforge/envs/bioml/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/meiri.yoav/mambaforge/envs/bioml/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(
/home/meiri.yoav/mambaforge/envs/bioml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value 

Validation: 0it [00:00, ?it/s]

/home/meiri.yoav/mambaforge/envs/bioml/lib/python3.11/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Runningstage.validating  ┃                           ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     val_test_subs_acc     │                           │    0.40760868787765503    │
│     val_test_subs_f1      │                           │    0.40760868787765503    │
│    val_test_subs_loss     │                           │    0.6975266933441162     │
│    val_train_subs_acc     │    0.4590163826942444     │                           │
│     val_train_subs_f1     │    0.4590163826942444     │                           │
│    val_train_subs_loss    │    0.6955657601356506     │                           │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘


  | Name     | Type               | Params
------------------------------------------------
0 | lstm     | LSTM               | 148 K 
1 | fc       | Sequential         | 8.4 K 
2 | accuracy | MulticlassAccuracy | 0     
3 | f1       | MulticlassF1Score  | 0     
4 | auc      | MulticlassAUROC    | 0     
------------------------------------------------
156 K     Trainable params
0         Non-trainable params
156 K     Total params
0.627     Total estimated model params size (MB)


...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
/home/meiri.yoav/mambaforge/envs/bioml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test_test_subs_acc     │                           │    0.7647058963775635     │
│     test_test_subs_f1     │                           │    0.7647058963775635     │
│    test_test_subs_loss    │                           │    0.5390118360519409     │
│    test_train_subs_acc    │    0.8666666746139526     │                           │
│    test_train_subs_f1     │    0.8666666746139526     │                           │
│   test_train_subs_loss    │    0.4334997534751892     │                           │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇█
test_test_subs_acc/dataloader_idx_1,▁
test_test_subs_f1/dataloader_idx_1,▁
test_test_subs_loss/dataloader_idx_1,▁
test_train_subs_acc/dataloader_idx_0,▁
test_train_subs_f1/dataloader_idx_0,▁
test_train_subs_loss/dataloader_idx_0,▁
train_acc,▁▆▇▇▇▇▇███
train_f1,▁▆▇▇▇▇▇███
train_loss,█▅▃▂▂▁▁▁▁▁
trainer/global_step,▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▇▇▇▇███
